In [1]:
import pandas as pd
import numpy as np
import nfl_data_py as nfl
pd.set_option('display.max_columns', None)

In [2]:
years = list(range(2016, 2023))

In [3]:
# could add epa if we want to determine what play type is best

In [4]:
cols = ["posteam", "posteam_type", "side_of_field", "yardline_100", "half_seconds_remaining", "game_seconds_remaining", "down",
       "ydstogo", "score_differential", "wp", "weather", "roof", "surface", "wind", "home_coach", "away_coach", "desc", "play_type", "pass_length", 
       "air_yards", "run_location", "drive", "yrdln"]

In [5]:
nfl_df = nfl.import_pbp_data(years, columns = cols, cache = False, alt_path= None)

2016 done.
2017 done.
2018 done.
2019 done.
2020 done.
2021 done.
2022 done.
Downcasting floats.


In [6]:
nfl_df.shape

(341697, 38)

In [7]:
nfl_df.head()

,posteam,posteam_type,side_of_field,yardline_100,half_seconds_remaining,game_seconds_remaining,down,ydstogo,score_differential,wp,weather,roof,surface,wind,home_coach,away_coach,desc,play_type,pass_length,air_yards,run_location,drive,yrdln,play_id,old_game_id,season,nflverse_game_id,possession_team,offense_formation,offense_personnel,defenders_in_box,defense_personnel,number_of_pass_rushers,players_on_play,offense_players,defense_players,n_offense,n_defense
0,None,None,None,NaN,1800.0,3600.0,NaN,0.0,NaN,0.546262,"Partly Cloudy Temp: 84° F, Humidity: 44%, Wind...",outdoors,grass,8.0,John Harbaugh,Rex Ryan,GAME,None,None,NaN,None,NaN,BUF 35,1.0,2016091101,2016,2016_01_BUF_BAL,,None,None,NaN,None,NaN,,,,0.0,0.0
1,BAL,home,BUF,35.0,1800.0,3600.0,NaN,0.0,0.0,0.546262,"Partly Cloudy Temp: 84° F, Humidity: 44%, Wind...",outdoors,grass,8.0,John Harbaugh,Rex Ryan,4-Gay kicks 65 yards from BUF 35 to BAL 0. 14-...,kickoff,None,NaN,None,1.0,BUF 35,36.0,2016091101,2016,2016_01_BUF_BAL,BUF,None,None,NaN,None,NaN,40078;40151;37977;36060;40494;40053;40253;4011...,00-0030041;00-0030073;00-0029799;00-0030433;00...,00-0029892;00-0029895;00-0028497;00-0027539;00...,10.0,11.0
2,BAL,home,BAL,86.0,1797.0,3597.0,1.0,10.0,0.0,0.536378,"Partly Cloudy Temp: 84° F, Humidity: 44%, Wind...",outdoors,grass,8.0,John Harbaugh,Rex Ryan,(14:57) 29-J.Forsett right end pushed ob at BA...,run,None,NaN,right,1.0,BAL 14,58.0,2016091101,2016,2016_01_BUF_BAL,BAL,I_FORM,"2 RB, 1 TE, 2 WR",8.0,"4 DL, 3 LB, 4 DB",NaN,38540;41302;40078;35553;38582;43295;40053;4336...,00-0029892;00-0027714;00-0032965;00-0029893;00...,00-0029542;00-0031171;00-0029566;00-0030073;00...,11.0,11.0
3,BAL,home,BAL,80.0,1772.0,3572.0,2.0,4.0,0.0,0.541975,"Partly Cloudy Temp: 84° F, Humidity: 44%, Wind...",outdoors,grass,8.0,John Harbaugh,Rex Ryan,"(14:32) (No Huddle, Shotgun) 5-J.Flacco pass s...",pass,short,-4.0,None,1.0,BAL 20,85.0,2016091101,2016,2016_01_BUF_BAL,BAL,SHOTGUN,"2 RB, 1 TE, 2 WR",6.0,"4 DL, 3 LB, 4 DB",3.0,41302;40078;38540;35553;43295;38582;40053;4336...,00-0029892;00-0027714;00-0032965;00-0029893;00...,00-0031171;00-0029542;00-0029566;00-0030073;00...,11.0,11.0
4,BAL,home,BAL,75.0,1741.0,3541.0,1.0,10.0,0.0,0.546262,"Partly Cloudy Temp: 84° F, Humidity: 44%, Wind...",outdoors,grass,8.0,John Harbaugh,Rex Ryan,"(14:01) (No Huddle, Shotgun) 29-J.Forsett left...",run,None,NaN,left,1.0,BAL 25,109.0,2016091101,2016,2016_01_BUF_BAL,BAL,SHOTGUN,"2 RB, 1 TE, 2 WR",7.0,"4 DL, 3 LB, 4 DB",NaN,40078;38540;41302;35553;43295;38582;40053;4336...,00-0029892;00-0027714;00-0032965;00-0029893;00...,00-0029542;00-0031171;00-0029566;00-0030073;00...,11.0,11.0


In [8]:
nfl_clean = nfl_df[(nfl_df['play_type'] == 'pass') | (nfl_df['play_type'] == 'run') |
                     (nfl_df['play_type'] == 'punt') | (nfl_df['play_type'] == 'field_goal')].reset_index(drop=True)

In [9]:
nfl_clean.shape

(262860, 38)

In [10]:
# make yardline yards to endzone

In [12]:
yards_to_end = []
for i in range(len(nfl_clean)):
    if nfl_clean['posteam'][i] == nfl_clean["yrdln"][i].split(" ")[0]:
        yards_to_end.append(100 - int(nfl_clean['yrdln'][i].split(" ")[1]))
    else:
        yards_to_end.append(nfl_clean['yrdln'][i].split(" ")[1])
        

In [13]:
nfl_clean['yds_to_endzone'] = yards_to_end

In [14]:
# check for rain

In [15]:
rain_type = ['Rain', 'Raining', 'Rainy', 'Showers']
weather_type=[]
for w in nfl_clean['weather']:
    if w:
        if w.split(" ")[0] in rain_type:
            weather_type.append("Rain")
        elif w.split(" ")[0] == "Snow":
            weather_type.append("Snow")
        else:
            weather_type.append("None")
    else:
        weather_type.append("None")

In [16]:
nfl_clean['Precipitation'] = weather_type

In [17]:
# head coach on offense

In [18]:
offense_hc = []
for i in range(len(nfl_clean)):
    if nfl_clean['posteam_type'][i] == 'home':
        offense_hc.append(nfl_clean['home_coach'][i])
    else:
        offense_hc.append(nfl_clean['away_coach'][i])


In [19]:
nfl_clean['Offense_HC'] = offense_hc

In [20]:
# replace missing roof with outdoors (None values)

In [21]:
nfl_clean['roof'] = nfl_clean['roof'].fillna("outdoors")

In [22]:
# use indoor or outdoor for roof values

In [23]:
indoor_type = ['closed', 'dome']
outdoor_type = ['open', 'outdoors']
indoors = []
for i in range(len(nfl_clean)):
    if nfl_clean['roof'][i] in indoor_type:
        indoors.append(True)
    else:
        indoors.append(False)
nfl_clean['Indoors'] = indoors

In [24]:
# drop unnecessary columns

In [25]:
nfl_clean1 = nfl_clean.drop(["posteam_type", "side_of_field", "yardline_100", "weather", "home_coach", "away_coach", "play_id", 
                            "old_game_id", "possession_team", "players_on_play", "offense_players",
                            "defense_players", "n_offense", "n_defense", "number_of_pass_rushers","yrdln", "roof"], axis = 1)

In [26]:
# replace NaN in wind with 0

In [27]:
nfl_clean1['wind'] = nfl_clean1['wind'].fillna(0)

In [28]:
# replace NaN in down with 1 (two point conversions so effectively not kicking situation)

In [29]:
nfl_clean1['down'] = nfl_clean1['down'].fillna(1.0)

In [32]:
nfl_clean2 = nfl_clean1.dropna(subset=['nflverse_game_id', 'drive'])

In [33]:
nfl_clean2.isna().sum()

posteam                        0
half_seconds_remaining         0
game_seconds_remaining         0
down                           0
ydstogo                        0
score_differential             0
wp                             0
surface                        0
wind                           0
desc                           0
play_type                      0
pass_length               130607
air_yards                 130578
run_location              164750
drive                          0
season                         0
nflverse_game_id               0
offense_formation          23519
offense_personnel          23204
defenders_in_box           23450
defense_personnel          23204
yds_to_endzone                 0
Precipitation                  0
Offense_HC                     0
Indoors                        0
dtype: int64

In [39]:
# simplify offense formation ( ex 12, 11 personnel)

In [40]:
import re

In [41]:
off_personnel = []
for p in nfl_clean2['offense_personnel']:
    if p:
        rb = re.search('. RB', p)[0]
        te = re.search('. TE', p)[0]
        off_personnel.append(rb.split(" ")[0] + te.split(" ")[0])
    else:
        off_personnel.append("Kick/Punt")

In [42]:
nfl_clean2['offense_personnel'] = off_personnel

<ipython-input-42-aabb76c64944>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nfl_clean2['offense_personnel'] = off_personnel


In [43]:
# simplify defense personnel (DL - LB)

In [44]:
def_personnel = []
for d in nfl_clean2['defense_personnel']:
    if d:
        dl = re.search('. DL', d)[0]
        lb = re.search('. LB', d)[0]
        def_personnel.append(dl.split(" ")[0] + "-" + lb.split(" ")[0])
    else:
        def_personnel.append("Kick/Punt")

In [45]:
nfl_clean2['defense_personnel'] = def_personnel

<ipython-input-45-5bef5ecfc5c1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nfl_clean2['defense_personnel'] = def_personnel


In [49]:
nfl_clean2['offense_formation'] = nfl_clean2['offense_formation'].fillna("Kick/Punt")
nfl_clean2["defenders_in_box"] = nfl_clean2['defenders_in_box'].fillna("Kick/Punt")

<ipython-input-49-be2a8212033b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nfl_clean2['offense_formation'] = nfl_clean2['offense_formation'].fillna("Kick/Punt")
<ipython-input-49-be2a8212033b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nfl_clean2["defenders_in_box"] = nfl_clean2['defenders_in_box'].fillna("Kick/Punt")


In [50]:
nfl_clean2.isna().sum()

posteam                        0
half_seconds_remaining         0
game_seconds_remaining         0
down                           0
ydstogo                        0
score_differential             0
wp                             0
surface                        0
wind                           0
desc                           0
play_type                      0
pass_length               130607
air_yards                 130578
run_location              164750
drive                          0
season                         0
nflverse_game_id               0
offense_formation              0
offense_personnel              0
defenders_in_box               0
defense_personnel              0
yds_to_endzone                 0
Precipitation                  0
Offense_HC                     0
Indoors                        0
dtype: int64

In [51]:
# could add previous play type
# sort by nfl_verse_game_id and drive

In [52]:
nfl_clean2[(nfl_clean2['nflverse_game_id'] == "2016_01_BUF_BAL") & (nfl_clean2['drive'] == 1.0)]

,posteam,half_seconds_remaining,game_seconds_remaining,down,ydstogo,score_differential,wp,surface,wind,desc,play_type,pass_length,air_yards,run_location,drive,season,nflverse_game_id,offense_formation,offense_personnel,defenders_in_box,defense_personnel,yds_to_endzone,Precipitation,Offense_HC,Indoors
0,BAL,1797.0,3597.0,1.0,10.0,0.0,0.536378,grass,8.0,(14:57) 29-J.Forsett right end pushed ob at BA...,run,None,NaN,right,1.0,2016,2016_01_BUF_BAL,I_FORM,21,8.0,4-3,86,None,John Harbaugh,False
1,BAL,1772.0,3572.0,2.0,4.0,0.0,0.541975,grass,8.0,"(14:32) (No Huddle, Shotgun) 5-J.Flacco pass s...",pass,short,-4.0,None,1.0,2016,2016_01_BUF_BAL,SHOTGUN,21,6.0,4-3,80,None,John Harbaugh,False
2,BAL,1741.0,3541.0,1.0,10.0,0.0,0.546262,grass,8.0,"(14:01) (No Huddle, Shotgun) 29-J.Forsett left...",run,None,NaN,left,1.0,2016,2016_01_BUF_BAL,SHOTGUN,21,7.0,4-3,75,None,John Harbaugh,False
3,BAL,1715.0,3515.0,2.0,10.0,0.0,0.528030,grass,8.0,(13:35) (No Huddle) 5-J.Flacco pass short righ...,pass,short,1.0,None,1.0,2016,2016_01_BUF_BAL,I_FORM,21,7.0,4-3,75,None,John Harbaugh,False
4,BAL,1674.0,3474.0,3.0,1.0,0.0,0.561710,grass,8.0,(12:54) (No Huddle) 28-T.West right tackle to ...,run,None,NaN,right,1.0,2016,2016_01_BUF_BAL,I_FORM,12,8.0,4-3,66,None,John Harbaugh,False
5,BAL,1612.0,3412.0,4.0,6.0,0.0,0.471003,grass,8.0,(11:52) (Punt formation) 4-S.Koch punts 39 yar...,punt,None,NaN,None,1.0,2016,2016_01_BUF_BAL,Kick/Punt,Kick/Punt,Kick/Punt,Kick/Punt,71,None,John Harbaugh,False


In [53]:
nfl_clean2.head()

,posteam,half_seconds_remaining,game_seconds_remaining,down,ydstogo,score_differential,wp,surface,wind,desc,play_type,pass_length,air_yards,run_location,drive,season,nflverse_game_id,offense_formation,offense_personnel,defenders_in_box,defense_personnel,yds_to_endzone,Precipitation,Offense_HC,Indoors
0,BAL,1797.0,3597.0,1.0,10.0,0.0,0.536378,grass,8.0,(14:57) 29-J.Forsett right end pushed ob at BA...,run,None,NaN,right,1.0,2016,2016_01_BUF_BAL,I_FORM,21,8.0,4-3,86,None,John Harbaugh,False
1,BAL,1772.0,3572.0,2.0,4.0,0.0,0.541975,grass,8.0,"(14:32) (No Huddle, Shotgun) 5-J.Flacco pass s...",pass,short,-4.0,None,1.0,2016,2016_01_BUF_BAL,SHOTGUN,21,6.0,4-3,80,None,John Harbaugh,False
2,BAL,1741.0,3541.0,1.0,10.0,0.0,0.546262,grass,8.0,"(14:01) (No Huddle, Shotgun) 29-J.Forsett left...",run,None,NaN,left,1.0,2016,2016_01_BUF_BAL,SHOTGUN,21,7.0,4-3,75,None,John Harbaugh,False
3,BAL,1715.0,3515.0,2.0,10.0,0.0,0.528030,grass,8.0,(13:35) (No Huddle) 5-J.Flacco pass short righ...,pass,short,1.0,None,1.0,2016,2016_01_BUF_BAL,I_FORM,21,7.0,4-3,75,None,John Harbaugh,False
4,BAL,1674.0,3474.0,3.0,1.0,0.0,0.561710,grass,8.0,(12:54) (No Huddle) 28-T.West right tackle to ...,run,None,NaN,right,1.0,2016,2016_01_BUF_BAL,I_FORM,12,8.0,4-3,66,None,John Harbaugh,False


In [70]:
len(nfl_clean2[nfl_clean2['play_type'] == 'punt'])

15928

In [71]:
len(nfl_clean2)

261623

In [63]:
# note: need to decide what to do for kick and punt drives as the formation will pretty much always give it away:
# either drop kick/punt or drop formation/personnel/defenders in box - Leaning towards dropping kicks

In [54]:
# input

In [55]:
nfl_input = nfl_clean2.drop(['posteam', 'desc', 'play_type', 'pass_length', 'air_yards', 'run_location', 
                            'season', 'nflverse_game_id'], axis = 1)

In [59]:
# output

In [60]:
nfl_output = nfl_clean2['play_type']

In [62]:
nfl_output.head(10)

0     run
1    pass
2     run
3    pass
4     run
5    punt
6    pass
7     run
8    pass
9    punt
Name: play_type, dtype: object

In [2]:
# try logistic regression, neural network, maybe RNN if we can implement previous plays on the drive
# not reinforcement - too many variables when it comes to roster to look at success of a play as a factor